In [ ]:
from extracao.reading import *
from extracao.main import format_matlab
from dotenv import load_dotenv

load_dotenv()

folder = Path.cwd().parent / 'dados'


In [ ]:
base = format_matlab(read_base(folder))

In [ ]:
base

,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,#1,0.028,-22.662779,-43.476391,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557670,J9E,8.0
1,#2,0.030,-23.709999,-46.273335,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1558412,J3E,2.0
2,#3,0.030,-23.441668,-46.590832,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557823,J3E,1.0
3,#4,0.030,-22.926666,-43.264999,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,859761,J3E,0.5
4,#5,0.030,-22.774166,-47.004444,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,859753,J3E,1.0
...,...,...,...,...,...,...,...,...,...
832361,#832362,148650.000,-2.509556,-44.294556,"[MOS] L, FB, Apil Seguranca E Vigilancia Ltda ...",19,1011671368,F1D,7.6
832362,#832363,4096125.000,-29.872139,-51.170582,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698585682,F7D,25.0
832363,#832364,4265625.000,-29.876944,-51.170692,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698585704,F7D,25.0
832364,#832365,4265625.000,-29.870222,-51.201305,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698801288,F7D,25.0


In [ ]:
aero = read_aero(folder)

In [ ]:
df = merge_close_rows(base.iloc[-100:], aero)

Output()

In [ ]:
df = pd.merge(base, aero, on=['Frequency'], how='outer')

In [ ]:
def distancia(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km

In [ ]:
condition = np.all(df[['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].notna(), axis=1)

In [ ]:
df.loc[condition, 'dist'] = df.loc[condition, ['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].apply(distancia , axis=1)

In [ ]:
condition = (df.dist.notna()) & (df.dist <= MAX_DIST) 
df.loc[condition, 'Description_x'] = df.loc[condition, 'Description_x'] + ' | ' + df.loc[condition, 'Description_y']  

In [ ]:
df.to_excel(folder / 'mesclados_outer_join.xlsx', index=False)

In [ ]:
aero.to_excel(folder / 'aero.xlsx', index=False)